# Inference mmdetection model from other format

## 1. Inference mmdetection pytorch model

In [47]:
from mmdet.apis import init_detector, inference_detector, async_inference_detector
from mmdet.utils import register_all_modules
import torch
import torchvision.transforms as transforms
from PIL import Image
import mmcv
import numpy as np

# 指定模型的配置文件和 checkpoint 文件路径
config_file = 'configs/yolo/yolov3_mobilenetv2_8xb24-ms-416-300e_coco.py'
checkpoint_file = 'work_dirs/yolov3_mobilenetv2_pretrained/best_pascal_voc_mAP_epoch_27.pth'

#Register all modules in mmdet into the registries
register_all_modules()
# 若检测到有GPU则使用GPU
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")

# 根据配置文件和 checkpoint 文件构建模型
model = init_detector(config_file, checkpoint_file, device=device)
model.eval()


# 定义预处理方法
preprocess = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# 加载图像并进行预处理
image = Image.open('../data/VOCdevkit/VOC2007/test_img/000001.jpg')
image_tensor = preprocess(image)
# 添加一个维度以匹配模型的输入
image_tensor = image_tensor.unsqueeze(0)
output = model(image_tensor)


img = mmcv.imread( '../data/VOCdevkit/VOC2007/test_img/000001.jpg', channel_order='rgb')
output2 = inference_detector(model, img)

output3 = torch.cat((output2.pred_instances['bboxes'], output2.pred_instances['labels'].unsqueeze(1), 
                     output2.pred_instances['scores'].unsqueeze(1)), dim=1).numpy()
output3 = np.rint(output3[:-1]).astype(np.int32)
print(output3)
print(output2)


Loads checkpoint by local backend from path: work_dirs/yolov3_mobilenetv2_pretrained/best_pascal_voc_mAP_epoch_27.pth
[[  1   2 348 496  14   1]
 [ 64 245 171 365  11   1]]
<DetDataSample(

    META INFORMATION
    scale_factor: (0.4475920679886686, 0.448)
    batch_input_shape: (224, 160)
    ori_shape: (500, 353)
    img_id: 0
    pad_shape: (224, 160)
    img_shape: (224, 158)
    img_path: None

    DATA FIELDS
    gt_instances: <InstanceData(
        
            META INFORMATION
        
            DATA FIELDS
            bboxes: tensor([], size=(0, 4))
            labels: tensor([], dtype=torch.int64)
        ) at 0x7f06cadc86a0>
    ignored_instances: <InstanceData(
        
            META INFORMATION
        
            DATA FIELDS
            bboxes: tensor([], size=(0, 4))
            labels: tensor([], dtype=torch.int64)
        ) at 0x7f06cadca8f0>
    pred_instances: <InstanceData(
        
            META INFORMATION
        
            DATA FIELDS
            bbox